In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator # not needed?
from sklearn.model_selection import train_test_split # not needed?
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Define input shape and number of classes
img_height, img_width = 512, 512
num_classes = 2

# Load the pre-trained ResNet50 model with ImageNet weights
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# Add new fully-connected layers for binary classification
x = Flatten()(resnet.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=resnet.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
data_dir = "../data"
shape = (512, 512)

In [ ]:
train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.4,
  subset="training",
  seed=123,
  image_size=shape,
  shuffle = True)

In [ ]:
val_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.4,
  subset="validation",
  seed=123,
  image_size=shape,
  shuffle = True)

In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)
val_ds = val_ds.skip((2*val_batches) // 3)

In [ ]:
# Preprocess the image data
# data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

# Split the data into training and testing sets
"""data_dir = "../data"
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'
train_filenames = tf.io.gfile.glob(train_dir + '/*/*')
test_filenames = tf.io.gfile.glob(test_dir + '/*/*')
train_labels = [int('dog' in filename) for filename in train_filenames]
test_labels = [int('dog' in filename) for filename in test_filenames]
train_filenames, val_filenames, train_labels, val_labels = train_test_split(train_filenames, train_labels, test_size=0.2, random_state=42)
"""
"""# Load the training data
train_data = data_generator.flow_from_list(
        train_filenames,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

# Load the validation data
validation_data = data_generator.flow_from_list(
        val_filenames,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

# Load the testing data
test_data = data_generator.flow_from_list(
        test_filenames,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')
"""

es = EarlyStopping(patience=2,restore_best_weights=True)

# Train the model
history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,
        verbose=1,
        callbacks=[es])

# Evaluate the model on the test data
#test_loss, test_acc = model.evaluate(test_data, verbose=1)
#print('Test accuracy:', test_acc)


"""
In this updated code, we use the ImageDataGenerator class to apply data augmentation to the image data. We set rotation_range to randomly rotate the images by up to 20 degrees, zoom_range to randomly zoom the images by up to 20%, and horizontal_flip to randomly flip the images horizontally.

We then load the training, validation, and testing data using the flow_from_list method of the data generator. We load the training data with data augmentation and the validation and testing data without data augmentation.

We then train the model with data augmentation using the fit method of the model. Finally, we evaluate the model on the test data using the
"""